In [ ]:
from token_creator import get_creds
get_creds()

In [ ]:
from gmail_agent import gmail_agent
from calendar_agent import Calendar_agent
from maps_agent import maps_agent

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition,InjectedState
from langchain_core.messages import (
    HumanMessage,
    ToolMessage,
    
)
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools.base import InjectedToolCallId


from typing_extensions import TypedDict
from typing import Annotated

import os
from dotenv import load_dotenv 

#get graph visuals
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod
import os

import gradio as gr


In [2]:
load_dotenv()
GOOGLE_API_KEY=os.getenv('google_api_key')

In [3]:
GEMINI_MODEL='gemini-2.0-flash'
llm = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model=GEMINI_MODEL, temperature=0.3)

In [4]:
mail_agent=gmail_agent(llm)
calendar_agent=Calendar_agent(llm)
maps_agent=maps_agent(llm)

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [7]:
@tool
def maps_manager(query:str):
    """tool to use to answer maps and location queries
    this tool can:
    find locations such as restorants, bowling alleys, museums and others
    display those locations's infos (eg. adress, name, url, price range)
    args: query - pass the email related queries directly here
    """
    response=maps_agent.chat(str(query))
    return response

In [8]:
@tool
def mail_manager(query:str):
    """Tool to use to answer any email related queries
    this tool can:
    refresh the inbox
    show the inbox
    get a specific mail's content to display
    create a draft of a new mail
    show the current draft
    send the draft
    list all the drafts
    args: query - pass the email related queries directly here
    """
    response=mail_agent.chat(str(query))
    return response

In [9]:
@tool
def calendar_manager(query:str):
    """tool to use to answere any calendar or schedule related queries
    this tool can:
    create recuring events
    create quick events
    refresh the calendar
    show the calendar
    args: query - pass the calendar related queries directly here
    """
    response=calendar_agent.chat(str(query))
    return response

In [14]:
class google_agent:
    def __init__(self,llm: any):
        self.agent=self._setup(llm)
    def _setup(self,llm):
   
        langgraph_tools=[mail_manager,calendar_manager,maps_manager]



        graph_builder = StateGraph(State)

        # Modification: tell the LLM which tools it can call
        llm_with_tools = llm.bind_tools(langgraph_tools)
        tool_node = ToolNode(tools=langgraph_tools)
        def chatbot(state: State):
            """ travel assistant that answers user questions about their trip.
            Depending on the request, leverage which tools to use if necessary."""
            return {"messages": [llm_with_tools.invoke(state['messages'])]}

        graph_builder.add_node("chatbot", chatbot)

        
        graph_builder.add_node("tools", tool_node)
        # Any time a tool is called, we return to the chatbot to decide the next step
        graph_builder.add_edge(START,'chatbot')
        graph_builder.add_edge("tools", "chatbot")
        graph_builder.add_conditional_edges(
            "chatbot",
            tools_condition,
        )
        memory=MemorySaver()
        graph=graph_builder.compile(checkpointer=memory)
        return graph
        

    def display_graph(self):
        return display(
                        Image(
                                self.agent.get_graph().draw_mermaid_png(
                                    draw_method=MermaidDrawMethod.API,
                                )
                            )
                        )
    def stream(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        input_message = HumanMessage(content=input)
        for event in self.agent.stream({"messages": [input_message]}, config, stream_mode="values"):
            event["messages"][-1].pretty_print()

    def chat(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        response=self.agent.invoke({'messages':HumanMessage(content=str(input))},config)
        return response['messages'][-1].content
    
    def get_state(self, state_val:str):
        config = {"configurable": {"thread_id": "1"}}
        return self.agent.get_state(config).values[state_val]

In [15]:
agent=google_agent(llm)

In [16]:
agent.stream('find cool cocktail bars in east village')

================================ Human Message =================================

find cool cocktail bars in east village
================================== Ai Message ==================================
Tool Calls:
  maps_manager (1d0b7e12-fff3-4f24-9f29-3dc794abae1c)
 Call ID: 1d0b7e12-fff3-4f24-9f29-3dc794abae1c
  Args:
    query: cool cocktail bars in east village
================================= Tool Message =================================
Name: maps_manager

Here are some cool cocktail bars in the East Village with their addresses and Google Maps URLs:

*   **Amor y Amargo**: 443 E 6th St, New York, NY 10009, USA ([https://maps.google.com/?cid=6100183703284545489](https://maps.google.com/?cid=6100183703284545489))
*   **96 Tears**: 110 Avenue A, New York, NY 10009, USA ([https://maps.google.com/?cid=14534992369157057087](https://maps.google.com/?cid=14534992369157057087))
*   **Death & Co East Village**: 433 E 6th St, New York, NY 10009, USA ([https://maps.google.com/?cid=17498

In [ ]:
mail_agent.get_state('messages')

In [17]:
def chatbot(input,history):
    response=agent.chat(input)
    return response


In [18]:
demo = gr.ChatInterface(chatbot, type="messages", autofocus=False)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
